<a href="https://colab.research.google.com/github/vtu28960-del/vtu28960/blob/main/Task_13_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Plotting Probabilities Using Matplotlib

import random
import itertools
import collections
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Define deck and card values
SUITS = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']
RANK_VALUES = {r: i for i, r in enumerate(RANKS)}
DECK = [(rank, suit) for suit in SUITS for rank in RANKS]

def card_value(card):
    return RANK_VALUES[card[0]]

# Step 2: Compute exact single-round probabilities
pairs = list(itertools.permutations(DECK, 2))
a_wins = b_wins = ties = 0

for a_card, b_card in pairs:
    if card_value(a_card) > card_value(b_card):
        a_wins += 1
    elif card_value(a_card) < card_value(b_card):
        b_wins += 1
    else:
        ties += 1

total_pairs = len(pairs)
single_round_probs = {
    "A_win": a_wins / total_pairs,
    "B_win": b_wins / total_pairs,
    "Tie": ties / total_pairs
}

# Step 3: Simulation functions
def play_one_game(num_rounds):
    deck = DECK.copy()
    random.shuffle(deck)
    score_A = score_B = 0
    for _ in range(num_rounds):
        if len(deck) < 2:
            deck = DECK.copy()
            random.shuffle(deck)
        a_card = deck.pop()
        b_card = deck.pop()
        if card_value(a_card) > card_value(b_card):
            score_A += 1
        elif card_value(a_card) < card_value(b_card):
            score_B += 1
    if score_A > score_B:
        return "A"
    elif score_B > score_A:
        return "B"
    else:
        return "TIE"

def estimate_overall_probs(num_rounds, trials=3000):
    outcomes = collections.Counter(play_one_game(num_rounds) for _ in range(trials))
    return {k: outcomes[k] / trials for k in ("A", "B", "TIE")}

# Step 4: Run simulations
rounds_range = range(1, 26)
trials = 3000
estimates = []
for r in rounds_range:
    result = estimate_overall_probs(r, trials)
    estimates.append({
        "Rounds": r,
        "P(A_overall_win)": result["A"],
        "P(B_overall_win)": result["B"],
        "P(Tie_overall)": result["TIE"]
    })
df = pd.DataFrame(estimates)

# Step 5: Plot results
plt.figure(figsize=(6, 4))
plt.bar(single_round_probs.keys(), single_round_probs.values())
plt.title("Single-Round Outcome Probabilities (Exact)")
plt.ylabel("Probability")
plt.ylim(0, 1)
plt.grid(axis='y', linestyle=':')
plt.show()

plt.figure(figsize=(8, 4))
plt.plot(df["Rounds"], df["P(A_overall_win)"], marker='o', label="Player A")
plt.plot(df["Rounds"], df["P(B_overall_win)"], marker='s', label="Player B")
plt.plot(df["Rounds"], df["P(Tie_overall)"], marker='^', label="Tie")
plt.title("Overall Outcome Probabilities vs. Number of Rounds")
plt.xlabel("Number of Rounds")
plt.ylabel("Probability")
plt.legend()
plt.grid(linestyle=':')
plt.show()

# Step 6: Histogram for 5 rounds
chosen_rounds = 5
margin_counts = collections.Counter()
for _ in range(4000):
    deck = DECK.copy()
    random.shuffle(deck)
    score_A = score_B = 0
    for _ in range(chosen_rounds):
        a_card = deck.pop()
        b_card = deck.pop()
        if card_value(a_card) > card_value(b_card):
            score_A += 1
        elif card_value(a_card) < card_value(b_card):
            score_B += 1
    margin_counts[score_A - score_B] += 1

margins = sorted(margin_counts.keys())
frequencies = [margin_counts[m] / 4000 for m in margins]

plt.figure(figsize=(6, 4))
plt.bar(margins, frequencies)
plt.title(f"Histogram of Win Margin (A - B) for {chosen_rounds} Rounds")
plt.xlabel("Win Margin")
plt.ylabel("Relative Frequency")
plt.grid(axis='y', linestyle=':')
plt.show()

# Step 7: Display results
print("Single-round exact probabilities:")
for k, v in single_round_probs.items():
    print(f"  {k}: {v:.4f}")

print("\nSummary estimates for selected rounds:")
for sel in [1, 5, 10, 25]:
    row = df[df["Rounds"] == sel].iloc[0]
    print(f"Rounds={sel}: P(A)={row['P(A_overall_win)']:.4f}, "
          f"P(B)={row['P(B_overall_win)']:.4f}, P(Tie)={row['P(Tie_overall)']:.4f}")
